In [ ]:
import json
from typing import List, Optional, Union

class Transaction:
    """Класс для представления одной транзакции"""
    def __init__(self, date: str, category: str, amount: float, description: str):
        self.date = date
        self.category = category
        self.amount = amount
        self.description = description

    def to_dict(self) -> dict:
        """Преобразует транзакцию в словарь"""
        return {
            "date": self.date,
            "category": self.category,
            "amount": self.amount,
            "description": self.description
        }

    @staticmethod
    def from_dict(data: dict) -> 'Transaction':
        """Создает объект транзакции из словаря"""
        return Transaction(
            date=data["date"],
            category=data["category"],
            amount=data["amount"],
            description=data["description"]
        )

class Ledger:
    """Класс для управления списком транзакций и вычисления баланса"""
    def __init__(self):
        self.transactions: List[Transaction] = []
        self.income = 0.0
        self.expense = 0.0

    def add_transaction(self, transaction: Transaction):
        """Добавляет транзакцию в список и обновляет соответствующую сумму"""
        self.transactions.append(transaction)
        if transaction.category.lower() == "доход":
            self.income += transaction.amount
        elif transaction.category.lower() == "расход":
            self.expense += transaction.amount

    def edit_transaction(self, index: int, new_transaction: Transaction):
        """Изменяет существующую транзакцию и корректирует баланс"""
        if 0 <= index < len(self.transactions):
            # Удаляем старую транзакцию из баланса
            old_transaction = self.transactions[index]
            if old_transaction.category.lower() == "доход":
                self.income -= old_transaction.amount
            elif old_transaction.category.lower() == "расход":
                self.expense -= old_transaction.amount

            # Добавляем обновленную транзакцию в баланс
            if new_transaction.category.lower() == "доход":
                self.income += new_transaction.amount
            elif new_transaction.category.lower() == "расход":
                self.expense += new_transaction.amount

            # Обновляем запись
            self.transactions[index] = new_transaction
        else:
            raise IndexError("Неверный индекс транзакции.")

    def search_transactions(self, date: Optional[str] = None, category: Optional[str] = None,
                            amount: Optional[Union[float, tuple]] = None) -> List[Transaction]:
        """Выполняет поиск транзакций по дате, категории и/или сумме"""
        result = self.transactions
        if date:
            result = [t for t in result if t.date == date]
        if category:
            result = [t for t in result if t.category == category]
        if isinstance(amount, tuple):
            result = [t for t in result if amount[0] <= t.amount <= amount[1]]
        elif amount:
            result = [t for t in result if t.amount == amount]
        return result

    def get_balance(self) -> dict:
        """Возвращает текущий баланс: доходы, расходы и итог"""
        return {"income": self.income, "expense": self.expense, "balance": self.income - self.expense}

    def load_data(self, file_path: str):
        """Загружает транзакции из JSON-файла и пересчитывает баланс"""
        try:
            with open(file_path, "r") as file:
                data = json.load(file)
                self.transactions = [Transaction.from_dict(item) for item in data]
                # Пересчитываем начальные доходы и расходы
                self.income = sum(t.amount for t in self.transactions if t.category.lower() == "доход")
                self.expense = sum(t.amount for t in self.transactions if t.category.lower() == "расход")
        except (FileNotFoundError, json.JSONDecodeError):
            print("Предупреждение: Не удалось загрузить данные. Создан новый файл.")

    def save_data(self, file_path: str):
        """Сохраняет текущий список транзакций в JSON-файл"""
        try:
            with open(file_path, "w") as file:
                json.dump([t.to_dict() for t in self.transactions], file, ensure_ascii=False, indent=4)
        except IOError:
            print("Ошибка: Не удалось сохранить данные в файл.")

def get_float_input(prompt: str) -> float:
    """Получает числовое значение с плавающей запятой от пользователя с обработкой ошибок"""
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("Ошибка: Введите действительное число.")

def main():
    """Главная функция, реализующая интерфейс командной строки"""
    ledger = Ledger()
    ledger.load_data("transactions.json")

    while True:
        print("\n1. Вывести баланс")
        print("2. Добавить запись")
        print("3. Редактировать запись")
        print("4. Поиск по записям")
        print("5. Сохранить и выйти")

        choice = input("\nВыберите действие: ")

        if choice == "1":
            balance = ledger.get_balance()
            print(f"\nБаланс: {balance['balance']}, Доходы: {balance['income']}, Расходы: {balance['expense']}")

        elif choice == "2":
            date = input("Дата (YYYY-MM-DD): ")
            category = input("Категория (доход/расход): ")
            amount = get_float_input("Сумма: ")
            description = input("Описание: ")
            ledger.add_transaction(Transaction(date, category, amount, description))
            print("Запись добавлена!")

        elif choice == "3":
            try:
                index = int(input("Введите индекс записи для редактирования: "))
                date = input("Новая дата (YYYY-MM-DD): ")
                category = input("Новая категория (доход/расход): ")
                amount = get_float_input("Новая сумма: ")
                description = input("Новое описание: ")
                ledger.edit_transaction(index, Transaction(date, category, amount, description))
                print("Запись обновлена!")
            except ValueError:
                print("Ошибка: Введите корректное целое число.")
            except IndexError:
                print("Ошибка: Неверный индекс.")

        elif choice == "4":
            date = input("Поиск по дате (оставьте пустым, чтобы пропустить): ")
            category = input("Поиск по категории (доход/расход, оставьте пустым, чтобы пропустить): ")
            amount_str = input("Поиск по сумме (точная или диапазон, например 100-200): ")
            amount = None
            if '-' in amount_str:
                try:
                    min_amt, max_amt = map(float, amount_str.split('-'))
                    amount = (min_amt, max_amt)
                except ValueError:
                    print("Ошибка: Некорректный формат диапазона.")
            elif amount_str:
                try:
                    amount = float(amount_str)
                except ValueError:
                    print("Ошибка: Некорректный ввод суммы.")

            results = ledger.search_transactions(date, category, amount)
            for i, t in enumerate(results):
                print(f"[{i}] Дата: {t.date}, Категория: {t.category}, Сумма: {t.amount}, Описание: {t.description}")

        elif choice == "5":
            ledger.save_data("transactions.json")
            print("Данные сохранены. Выход.")
            break

        else:
            print("Ошибка: неверный выбор.")

if __name__ == "__main__":
    main()


Предупреждение: Не удалось загрузить данные. Создан новый файл.

1. Вывести баланс
2. Добавить запись
3. Редактировать запись
4. Поиск по записям
5. Сохранить и выйти

Выберите действие: 1

Баланс: 0.0, Доходы: 0.0, Расходы: 0.0

1. Вывести баланс
2. Добавить запись
3. Редактировать запись
4. Поиск по записям
5. Сохранить и выйти

Выберите действие: 2
Дата (YYYY-MM-DD): 2024-05-05
Категория (доход/расход): доход
Сумма: 30000
Описание: зарплата
Запись добавлена!

1. Вывести баланс
2. Добавить запись
3. Редактировать запись
4. Поиск по записям
5. Сохранить и выйти

Выберите действие: 4
Поиск по дате (оставьте пустым, чтобы пропустить): 2024-05-05
Поиск по категории (доход/расход, оставьте пустым, чтобы пропустить): 
Поиск по сумме (точная или диапазон, например 100-200): 
[0] Дата: 2024-05-05, Категория: доход, Сумма: 30000.0, Описание: зарплата

1. Вывести баланс
2. Добавить запись
3. Редактировать запись
4. Поиск по записям
5. Сохранить и выйти

Выберите действие: 3
Введите индекс запи